# Import

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import random
from typing import List, Union

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from sklearn.model_selection import KFold

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

import datetime
import pytz
import matplotlib.pyplot as plt

import utils
import model

# Config

In [ ]:
config = {
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'data_path': 'train1000', # change relative path of data -> ex) kaggle: '/kaggle/input/swdacon', colab: '/content/drive/MyDrive/open'
    'save_path': 'result', # change relative path where result is stored -> ex) kaggle: '/kaggle/working/', colab: '/content/drive/MyDrive/open'
    'train_data': 'train_1000.csv', # change train data csv name
    'test_data': 'test_20.csv', # change test data csv name
    'seed': 42,
    'valid_size': 0.1,
    'early_stopping': 30,
    'scheduler': True,
    'nums_pixel': False,
    'k-fold': 1, # 1이하의 정수이면 사용하지 않음
    'train' : {
       'batch_size' : 16,
       'num_workers': 1,
       'epochs': 200,
       'lr': 0.001,
    },
    'inference' : {
       'batch_size' : 8,
       'num_workers': 1,
       'threshold': 0.35,
    },
}

In [ ]:
if not os.path.exists(config['save_path']):
    os.makedirs(config['save_path'])
    print("save path가 생성되었습니다.")
print("save_path:", os.path.abspath(config['save_path']))
if not os.path.exists(config['data_path']):
    raise FileNotFoundError("base path가 존재하지 않습니다. 데이터가 있는 경로를 확인해주세요.")
else:
    print("data_path:", os.path.abspath(config['data_path']))

# Train
- 1024 * 1024 * 3 이미지 학습

### load train data

In [ ]:
train_df = pd.read_csv(f"{config['data_path']}/{config['train_data']}")
train_df['img_path'] = config['data_path'] + train_df['img_path'].str[1:]
if config['k-fold'] > 1:
    kfold = KFold(n_splits=config['k-fold'], random_state=config['seed'], shuffle=True)
    print(kfold)
else:
    train, val = train_test_split(train_df, test_size=config['valid_size'], random_state=config['seed'])
    print("train: ", len(train), "   valid: ", len(val))

### Custom Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_paths, mask_rles = None, transform=None, infer=False):
        self.img_paths = img_paths
        self.mask_rles = mask_rles
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths.iloc[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        mask_rle = self.mask_rles.iloc[idx]
        mask = utils.rle_decode(mask_rle, (image.shape[0], image.shape[1]))

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

### Data Loader

In [ ]:
utils.fix_seed(config['seed'])

if config['k-fold'] > 1:
    print(f"k-fold: {config['k-fold']} 입력되어, training할 때 fold별 dataloader가 생성됩니다.")
else:
    train_dataset = CustomDataset(img_paths=train['img_path'], mask_rles=train['mask_rle'], transform=utils.base_transform['train'])
    train_dataloader = DataLoader(train_dataset, batch_size=config['train']['batch_size'], shuffle=True)

    valid_dataset = CustomDataset(img_paths=val['img_path'], mask_rles=val['mask_rle'], transform=utils.base_transform['valid'])
    valid_dataloader = DataLoader(valid_dataset , batch_size=config['train']['batch_size'], shuffle=False)

### Model
- default: smp unet

### Load model

In [ ]:
if config['k-fold'] > 1:
    print(f"k-fold: {config['k-fold']} 입력되어, training할 때 fold별 model이 생성됩니다.")
else:
    model = model.SMP() # 사용할 모델 선택
    model = model.load_models("UNet")
    temp_model = model

### Validation

In [ ]:
def validation(config, model, criterion, valid_loader, val):
    model.eval()
    valid_loss = 0
    result = []
    transformed_mask = []
    val_df = val.copy()

    with torch.no_grad():
        for images, masks in tqdm(valid_loader):
            if type(transformed_mask) == torch.Tensor:
                transformed_mask = torch.cat([transformed_mask, masks])
            else:
                transformed_mask = masks.clone().detach()
            images = images.float().to(config['device'])
            masks = masks.float().to(config['device'])

            outputs = model(images)
            loss = criterion(outputs, masks.unsqueeze(1))
            valid_loss += loss.item()

            output_masks = torch.sigmoid(outputs).cpu().numpy()
            output_masks = np.squeeze(output_masks, axis=1)
            output_masks = (output_masks > config['inference']['threshold']).astype(np.uint8)

            for i in range(len(images)):
                mask_rle = utils.rle_encode(output_masks[i])
                if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                    result.append(-1)
                else:
                    result.append(mask_rle)
        val_df['valid_mask_rle'] = result
        val_df['transformed_mask_rle'] = list(map(utils.rle_encode, transformed_mask.squeeze().numpy()))
        dice_score = utils.calculate_dice_scores(val_df)
        if config['nums_pixel']:
            mean_error_ratio, more_pred, less_pred = utils.calculate_nums_pixel(val_df)
    if config['nums_pixel']:
        return valid_loss/len(valid_loader), dice_score, mean_error_ratio, more_pred, less_pred
    else:
        return valid_loss/len(valid_loader), dice_score

### Train

In [ ]:
def training(config, model, train_loader, valid_loader, val, fold=0):
    device = config['device']
    model = model.to(device)
    es_count = 0
    min_val_loss = float('inf')
    best_model = None

    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config['train']['lr'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-8, verbose=True)
    print("***TRAINING START***")
    # training loop
    for epoch in range(config['train']['epochs']):
        model.train()
        epoch_loss = 0
        for images, masks in tqdm(train_loader):
            images = images.float().to(config['device'])
            masks = masks.float().to(config['device'])

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks.unsqueeze(1))
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        if config['nums_pixel']:
            val_loss, dice_score, mean_error_ratio, more_pred, less_pred  = validation(config, model, criterion, valid_loader, val)
        else:
            val_loss, dice_score = validation(config, model, criterion, valid_loader, val)

        es_count += 1
        if min_val_loss > val_loss:
            es_count = 0
            min_val_loss = val_loss
            best_model = model
            state_dict = model.state_dict()
            best_epoch = epoch + 1
            print(f"Epoch [{epoch + 1}] New Minimum Valid Loss!")
            if epoch+1 > 50: # 비정상적인 종료에 대비해 몇 epoch 이상부터 저장할지 결정합니다.
                if fold:
                    current_model = f"fold{fold}_current_best_model.pt"
                else:
                    current_model = "current_best_model.pt"
                print("..save current best model..")
                torch.save(state_dict, f'{config["save_path"]}/{current_model}')

        if config['scheduler']:
            scheduler.step(val_loss)

        if es_count == config['early_stopping']:
            if config['nums_pixel']:
                print(f'Epoch {epoch+1}, Train Loss: {(epoch_loss/len(train_loader)):6f}, Valid Loss: {val_loss:6f}, ES Count: {es_count}')
                print(f'Dice Coefficient: {dice_score:6f}, (GT - Pred)/GT: {mean_error_ratio:2f}, More Pred : Less Pred = {more_pred} : {less_pred}')
            else:
                print(f'Epoch {epoch+1}, Train Loss: {(epoch_loss/len(train_loader)):6f}, Valid Loss: {val_loss:6f}, Dice Coefficient: {dice_score:6f}, ES Count: {es_count}')
            print(f"Early Stopping Count에 도달했습니다! \nEarly Stopping Count: {config['early_stopping']} Best Epoch: {best_epoch}")
            print("***TRAINING DONE***")
            return best_model, state_dict

        if config['nums_pixel']:
            print(f'Epoch {epoch+1}, Train Loss: {(epoch_loss/len(train_loader)):6f}, Valid Loss: {val_loss:6f}, ES Count: {es_count}')
            print(f'Dice Coefficient: {dice_score:6f}, (GT - Pred)/GT: {mean_error_ratio:2f}, More Pred : Less Pred = {more_pred} : {less_pred}')
        else:
            print(f'Epoch {epoch+1}, Train Loss: {(epoch_loss/len(train_loader)):6f}, Valid Loss: {val_loss:6f}, Dice Coefficient: {dice_score:6f}, ES Count: {es_count}')
        print("------------------------------------------------------------------------------------")

    print(f"Early Stopping Count에 도달하지 않았습니다! \nEarly Stopping Count: {config['early_stopping']} Best Epoch: {best_epoch}")
    print("***TRAINING DONE***")
    return best_model, state_dict

In [ ]:
torch.cuda.is_available() # 학습 전에 GPU 쓰고 있나 확인

In [ ]:
utils.fix_seed(config['seed'])

if config['k-fold'] > 1:
    best_models, best_model_state_dicts = [], []
    for i, (train_idx, val_idx) in enumerate(kfold.split(train_df)):
        train = train_df.iloc[train_idx,:]
        val = train_df.iloc[val_idx,:]
        train_dataset = CustomDataset(img_paths=train['img_path'], mask_rles=train['mask_rle'], transform=custom_transform['train'])
        train_dataloader = DataLoader(train_dataset, batch_size=config['train']['batch_size'], shuffle=True)

        valid_dataset = CustomDataset(img_paths=val['img_path'], mask_rles=val['mask_rle'], transform=custom_transform['valid'])
        valid_dataloader = DataLoader(valid_dataset , batch_size=config['train']['batch_size'], shuffle=False)
        print(f"--- Start Fold {i + 1} ---")
        model = model.SMP() # 사용할 모델 선택
        model = model.load_models("UNet")
        temp_model = model
        best_model, best_model_state_dict = training(config, model, train_dataloader, valid_dataloader, val, i+1)
        best_models.append(best_model)
        best_model_state_dicts.append(best_model)
        print(f"---- End Fold {i + 1} ----")
else:
    best_model, best_model_state_dict = training(config, model, train_dataloader, valid_dataloader, val)

## Inference
- 224 * 224 * 3 이미지 추론

### load test data

In [ ]:
test_df = pd.read_csv(f"{config['data_path']}/{config['test_data']}")
test_df['img_path'] = config['data_path'] + test_df['img_path'].str[1:]

### Data Loader

In [ ]:
utils.fix_seed(config['seed'])

if config['k-fold'] > 1:
    print(f"k-fold: {config['k-fold']} 입력되어, inference할 때 fold별 dataloader가 생성됩니다.")
else:
    test_dataset = CustomDataset(img_paths=test_df['img_path'], transform=utils.base_transform['test'], infer=True)
    test_dataloader = DataLoader(test_dataset, batch_size=config['inference']['batch_size'], shuffle=False)

In [ ]:
def inference(config, model, test_loader):
    with torch.no_grad():
        model.eval()
        result = []
        for images in tqdm(test_loader):
            images = images.float().to(config['device'])

            outputs = model(images)
            masks = torch.sigmoid(outputs).cpu().numpy()
            masks = np.squeeze(masks, axis=1)
            masks = (masks > config['inference']['threshold']).astype(np.uint8)

            for i in range(len(images)):
                mask_rle = utils.rle_encode(masks[i])
                if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                    result.append(-1)
                else:
                    result.append(mask_rle)
    return result

In [ ]:
if config['k-fold'] > 1:
    inference_results = []
    for i, fold_model in enumerate(best_models):
        test_dataset = CustomDataset(img_paths=test_df['img_path'], transform=utils.base_transform['test'], infer=True)
        test_dataloader = DataLoader(test_dataset, batch_size=config['inference']['batch_size'], shuffle=False)
        print(f"--- Start Fold {i + 1} ---")
        inference_result = inference(config, fold_model, test_dataloader)
        inference_results.append(inference_result)
else:
    inference_result = inference(config, best_model, test_dataloader)

## Submission

In [ ]:
kst = pytz.timezone('Asia/Seoul')
now = datetime.datetime.now(tz=kst)
current_time = now.strftime("%y%m%d-%H_%M_%S")

if config['k-fold'] > 1:
    for i,inference_result in enumerate(inference_results):
        submit = pd.read_csv(f"{config['data_path']}/sample_submission.csv")
        submit['mask_rle'] = inference_result
        file_name = f"fold_{i+1}_{current_time}.csv"
        submit.to_csv(f"{config['save_path']}/{file_name}", index=False)

    # 모델 저장
    for i,best_model_state_dict in enumerate(best_model_state_dicts):
        model_name = f"fold_{i+1}_{current_time}.pt"
        torch.save(best_model, f"{config['data_path']}/{model_name}")
else:
    submit = pd.read_csv(f"{config['data_path']}/sample_submission.csv")
    submit['mask_rle'] = inference_result
    file_name = f"{current_time}.csv"
    submit.to_csv(f"{config['save_path']}/{file_name}", index=False)

    # 모델 저장
    model_name = f"{current_time}.pt"
    torch.save(best_model, f"{config['save_path']}/{model_name}")
print(file_name)
print(model_name)

# Submission Viewer
- fold 사용시엔 사용할 필요 x

In [ ]:
def submission_viewer(test_csv, submit_csv, img_num):
    """
    white -> 건물 black -> 배경
    1. Local에서 사용 시 test_csv, submit_csv, img_num만 입력
    2. colab에서 사용 시 아래 사항을 입력
    base_path = colab_base
    is_colab = True
    """
    mask_rle = submit_csv.iloc[img_num, 1]
    image_path = test_csv.iloc[img_num, 1]
    image = cv2.imread(image_path)
    mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))
    fig = plt.figure()
    ax1 = fig.add_subplot(1,2,1)
    ax1.imshow(image)
    ax1.set_title('image')
    ax2 = fig.add_subplot(1,2,2)
    ax2.imshow(mask,cmap='gray')
    ax2.set_title('mask')
    plt.show()

In [ ]:
if config['k-fold'] > 1:
    print("Ensemble을 진행하십시오.")
else:
    last_submit = pd.read_csv(f"{config['save_path']}/{file_name}")
    submission_viewer(test_df, last_submit, 0)
    submission_viewer(test_df, last_submit, 1)
    submission_viewer(test_df, last_submit, 2)

# inference 확인

In [ ]:
# 모델 불러와 파라미터 입력
temp_model.load_state_dict(torch.load('/home/kimyw/SWdacon2/SWdacon/open/fold1_epoch227_current_model.pt')) # pt 파일 경로 설정
# eval 모드
temp_model = temp_model.eval()
# GPU로 옮기기
temp_model = temp_model.to(config['device'])

In [ ]:

pred_dataset = CustomDataset(img_paths=train_df['img_path'], transform=utils.base_transform['valid'], infer=True)
pred_dataloader = DataLoader(pred_dataset, batch_size=config['inference']['batch_size'], shuffle=False)
inference_result = inference(config, temp_model, pred_dataloader)

K = A.augmentations.crops.transforms.CenterCrop(224,224,p=1.0)

submit = pd.DataFrame({ 'mask_rle' : inference_result})

In [ ]:
# 예측 결과 뷰어
def pred_viewer(train_df, valid_csv, img_num):
    """
    white -> 건물 black -> 배경
    1. Local에서 사용 시 train_df, pred_csv, img_num만 입력
    2. colab에서 사용 시 아래 사항을 입력
    data_path = colab_base
    is_colab = True
    """
    mask_rle = train_df.iloc[img_num, 2]
    image_path = train_df.iloc[img_num, 1]
    pred = valid_csv.iloc[img_num, 0]

    image = cv2.imread(image_path)
    image=K(image=image)['image']
    mask = utils.rle_decode(mask_rle, (1024, 1024))
    mask=K(image=mask)['image']
    pred = utils.rle_decode(pred, (224, 224))
    
    fig = plt.figure()
    ax1 = fig.add_subplot(1,3,1)
    ax1.imshow(image)
    ax1.set_title('image')
    ax2 = fig.add_subplot(1,3,2)
    ax2.imshow(pred,cmap='gray')
    ax2.set_title('pred_mask')
    ax3 = fig.add_subplot(1,3,3)
    ax3.imshow(mask,cmap='gray')
    ax3.set_title('gt_mask')
    plt.show()

In [ ]:
pred_viewer(train_df, submit, 18)